# Étape 3 – Modélisez et expérimentez avec plusieurs algorithmes

Dans cette étape, nous allons entraîner plusieurs modèles de classification, comparer leurs performances sur des métriques classiques et métier, et tracer toutes nos expérimentations avec MLflow. L’objectif est de construire une première version robuste du pipeline d’apprentissage automatique, intégrant la validation croisée, la gestion du déséquilibre des classes et la traçabilité des modèles.

### 1. Chargement des données pré-traitées

Nous chargeons ici les jeux de données `X_train`, `X_test`, `y_train`, `y_test` préparés à l’étape précédente. Il est important de valider que les formats sont corrects et de vérifier la distribution des classes, notamment si le dataset est déséquilibré.

In [ ]:
import pandas as pd

# Charger les données préparées depuis le notebook précédent
X_train = pd.read_parquet("../data/output/X_train.parquet")
X_test = pd.read_parquet("../data/output/X_test.parquet")
y_train = pd.read_parquet("../data/output/y_train.parquet").squeeze()
y_test = pd.read_parquet("../data/output/y_test.parquet").squeeze()

### 2. Mise en place d’une stratégie de validation croisée

Nous définissons une stratégie de validation robuste avec `StratifiedKFold`, afin de garantir une répartition homogène des classes cibles dans les splits d’entraînement et de validation. Cela permet de mieux évaluer la généralisation des modèles, en particulier dans le cas de classes déséquilibrées.


In [ ]:
from sklearn.model_selection import StratifiedKFold

# Définition d'une validation croisée stratifiée à 5 plis
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

### 3. Entraînement de modèles de base (Logistic Regression & Random Forest)

Nous commençons par entraîner des modèles simples mais efficaces : 
- **Régression logistique**, avec pénalisation des classes minoritaires.
- **Forêt aléatoire**, avec des hyperparamètres de base.

Chaque modèle sera évalué à l’aide de la validation croisée et ses performances seront tracées dans MLflow (paramètres, scores, modèle).


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Dictionnaire des modèles à tester
# Commented : 
# Logistic Regression: F1-score moyen = 0.1866 (+/- 0.0131)
# Random Forest: F1-score moyen = 0.2780 (+/- 0.0157)
# SVM: F1-score moyen = 0.1699 (+/- 0.0059)
models = {
#    "Logistic Regression": LogisticRegression(max_iter=500, class_weight='balanced'),
#    "Random Forest": RandomForestClassifier(n_estimators=100, max_depth=5, class_weight='balanced'),
#    "SVM": SVC(probability=True, class_weight='balanced'),
    "XGBoost": XGBClassifier(eval_metric="logloss", scale_pos_weight=10, random_state=42),
    "LightGBM": LGBMClassifier(class_weight="balanced", verbose=-1)
}

### 4. Test de modèles plus puissants (Boosting, MLP…)

Nous expérimentons ici des modèles plus avancés comme :
- **XGBoostClassifier**
- **LightGBMClassifier**
- (optionnel) **MLPClassifier**

Ces modèles sont généralement plus performants mais demandent une gestion plus fine des hyperparamètres et du surapprentissage.


In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import make_scorer, f1_score

def clean_column_names(df):
    df.columns = df.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)
    return df

X_train = clean_column_names(X_train)
X_test = clean_column_names(X_test)

X_sample = X_train.sample(n=5000, random_state=42)
y_sample = y_train.loc[X_sample.index]

# LightGBM ne supporte pas les colonnes avec caractères spéciaux → on les nettoie
X_sample.columns = X_sample.columns.str.replace('[^A-Za-z0-9_]+', '_', regex=True)

# Score basé sur le F1-score
scorer = make_scorer(f1_score, average="binary", pos_label=1)

# Évaluer les modèles avec validation croisée
for name, model in models.items():
    scores = cross_val_score(model, X_sample, y_sample, cv=cv, scoring=scorer)
    print(f"{name}: F1-score moyen = {scores.mean():.4f} (+/- {scores.std():.4f})")

### 5. Gestion du déséquilibre des classes

Nous intégrons ici deux approches complémentaires :
- La pondération des classes via `class_weight="balanced"`
- Le sur-échantillonnage de la classe minoritaire avec **SMOTE**

Nous comparons les résultats obtenus avec et sans traitement du déséquilibre, et analysons leur impact sur les scores métiers (recall, f1).


In [ ]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature

mlflow.set_tracking_uri("file:../mlruns")
mlflow.set_experiment("modele_classification")

for name, model in models.items():
    with mlflow.start_run(run_name=name):
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        score = f1_score(y_test, y_pred)
        f1_train = f1_score(y_train, model.predict(X_train))

        mlflow.log_param("model", name)
        mlflow.log_metric("f1_train", f1_train)
        mlflow.log_metric("f1_test", score)

        signature = infer_signature(X_test, y_pred)
        mlflow.sklearn.log_model(model, name.replace(" ", "_").lower(), signature=signature)

### 6. Enregistrement des expérimentations avec MLflow

Chaque run est tracé avec MLflow :
- Paramètres (`log_param`)
- Métriques (`log_metric`)
- Modèle (`log_model`)
- Métadonnées utiles (`set_tags`)

Nous rendons ainsi chaque expérimentation traçable, comparable, et potentiellement réutilisable dans une chaîne d’industrialisation.


In [ ]:
import mlflow
import mlflow.sklearn
from mlflow.models.signature import infer_signature
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
import joblib

# Paramètres
best_model = XGBClassifier(n_estimators=100, max_depth=5, use_label_encoder=False, eval_metric="logloss")
best_model.fit(X_train, y_train)

# Prédictions
y_pred = best_model.predict(X_test)
score = f1_score(y_test, y_pred)

# Tracking MLflow
with mlflow.start_run(run_name="XGBoost_best_model"):
    # Log des hyperparamètres
    mlflow.log_param("n_estimators", 100)
    mlflow.log_param("max_depth", 5)
    mlflow.log_param("model_type", "XGBoost")

    # Log des métriques
    mlflow.log_metric("f1_score", score)

    # Signature d'entrée/sortie
    signature = infer_signature(X_test, y_pred)

    # Log du modèle
    mlflow.sklearn.log_model(best_model, "xgboost_best_model", signature=signature)

    # Optionnel : ajout de tags pour traçabilité
    mlflow.set_tags({
        "stage": "final_model",
        "author": "David Worsley-Tonks",
        "model": "XGBoost",
        "version": "v1"
    })

# Sauvegarde locale
joblib.dump(best_model, "../models/best_model_xgb.pkl")

### 7. Comparaison des performances et sélection de modèle

Nous rassemblons ici les scores clés (accuracy, recall, f1, AUC) de tous les modèles testés. Une comparaison objective est présentée sous forme de tableau. L’objectif est de sélectionner le ou les modèles les plus adaptés pour la suite du projet.


In [ ]:
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score

# Comparaison des performances des modèles sur le test set
scores = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

    scores.append({
        "Modèle": name,
        "Accuracy": accuracy_score(y_test, y_pred),
        "Recall": recall_score(y_test, y_pred),
        "F1-score": f1_score(y_test, y_pred),
        "AUC": roc_auc_score(y_test, y_proba) if y_proba is not None else None
    })

import pandas as pd

df_scores = pd.DataFrame(scores).sort_values(by="F1-score", ascending=False)
display(df_scores)

### 8. Exploration des résultats dans l’interface MLflow

L’interface MLflow (accessible via `http://localhost:8889`) permet de :
- Visualiser l’historique des runs
- Comparer les performances
- Retrouver les paramètres et modèles associés
- Faciliter le versioning des expérimentations

In [ ]:
import webbrowser

# Ouvrir automatiquement l’interface MLflow (en local)
webbrowser.open("http://localhost:8889")

### 9. Conclusion

Au cours de cette étape, nous avons comparé plusieurs familles de modèles de classification supervisée, notamment Random Forest, XGBoost et LightGBM. Nous avons mis en place une stratégie rigoureuse de validation croisée pour comparer les modèles sur des métriques adaptées à notre problème déséquilibré (notamment le F1-score).

Nous avons également intégré MLflow afin de tracer chaque expérimentation de manière transparente : enregistrement des paramètres, scores et modèles dans une interface dédiée. Cette démarche permet de faciliter l’analyse comparative des performances, de suivre l’évolution des expérimentations, et de poser une base solide pour les étapes suivantes.

Les résultats montrent que XGBoost obtient les meilleures performances globales, avec un F1-score, une précision et un rappel élevés sur notre jeu de test.

Grâce à cette étape, nous avons construit un socle robuste pour la suite du projet : amélioration des modèles, tuning d’hyperparamètres, interprétabilité, et industrialisation.